In [1]:
# install Pyomo and solvers
import requests
import types

url = "https://raw.githubusercontent.com/jckantor/MO-book/main/python/helper.py"
helper = types.ModuleType("helper")
exec(requests.get(url).content, helper.__dict__)

helper.install_pyomo()
helper.install_glpk()
helper.install_cbc()
helper.install_ipopt()

pyomo was previously installed
glpk was previously installed
cbc was previously installed
ipopt was previously installed


True

In [2]:
%matplotlib inline
import pyomo.environ as pyo
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import logging
from IPython.display import Markdown

cbc_solver = pyo.SolverFactory('cbc')
glpk_solver = pyo.SolverFactory('glpk')
ipopt_solver = pyo.SolverFactory('ipopt')

# News vendor problem 

Consider the news vendor problem seen at lecture and suppose that. Suppose the unit prices are $c = 10 , s = 25 , r = 5$, and that the newspaper demand $\xi$ modelled as a continuous random variable.

(a) Find the optimal solution of the news vendor problem using the explicit formula that features the inverse CDFs/quantile functions for the following three distributions:
 - a uniform distribution on the interval $[50,150]$;
*Hint: see [Uniform distribution CDF and its inverse](https://en.wikipedia.org/wiki/Continuous_uniform_distribution#Cumulative_distribution_function)*
 - a Pareto distribution on the interval $[50,+\infty)$ with $x_m=50$ and exponent $\alpha=2$. *Hint: the inverse CDF for a Pareto distribution is given by* $H^{-1}(\varepsilon) = \frac{x_m}{(1-\varepsilon)^{1/\alpha}}$.
 - a Weibull distribution on the interval $[0,+\infty)$ with shape parameter $k=2$ and scale parameter $\lambda=113$, see [Weibull distribution CDF and its inverse](https://en.wikipedia.org/wiki/Weibull_distribution#Cumulative_distribution_function).


In [3]:
# Setting the parameters
c = 10
s = 25
r = 5

# Definining the three inverse CDFs/quantile functions for the three distributions
def quantileuniform(epsilon, a, b):
    return a + epsilon*(b-a)

def quantilepareto(epsilon, xm, alpha):
    return xm/(1.0-epsilon)**(1.0/alpha)

def quantileweibull(epsilon, k, l):
    return l*(-np.log(1-epsilon))**(1.0/k)

# Calculating the optimal decision for each of the three distributions
display(Markdown(f"**Optimal solution** for uniform distribution: ${quantileuniform((s-c)/(s-r),50,150):.2f}$"))
display(Markdown(f"**Optimal solution** for Pareto distribution: ${quantilepareto((s-c)/(s-r),50,2):.2f}$"))
display(Markdown(f"**Optimal solution** for Weibull distribution: ${quantileweibull((s-c)/(s-r),2,113):.2f}$"))

**Optimal solution** for uniform distribution: $125.00$

**Optimal solution** for Pareto distribution: $100.00$

**Optimal solution** for Weibull distribution: $133.05$

Note that all the three distribution above have the same expected value, that is $\mathbb E \xi = 100$.

(b) Find the optimal solution of the deterministic LP model obtained by assuming the demand is fixed $\xi=\bar{\xi}$ and equal to the average demand $\bar{\xi} = \mathbb E \xi = 100$.

In [4]:
# Two-stage stochastic LP

c = 10
s = 25
r = 5

model = pyo.ConcreteModel()
model.xi = 100

# first stage variable
model.x = pyo.Var(within=pyo.NonNegativeReals) #bought

def first_stage_profit(model):
    return -c * model.x

model.first_stage_profit = pyo.Expression(rule=first_stage_profit)

# second stage variables
model.y = pyo.Var(within=pyo.NonNegativeReals) #sold
model.z = pyo.Var(within=pyo.NonNegativeReals) #unsold to be returned 

# second stage constraints
model.cantsoldthingsidonthave = pyo.Constraint(expr=model.y <= model.xi)
model.newspapersdonotdisappear = pyo.Constraint(expr=model.y + model.z == model.x)

def second_stage_profit(model):
    return s * model.y + r * model.z

model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

def total_profit(model):
    return model.first_stage_profit + model.second_stage_profit

model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

result = cbc_solver.solve(model)

display(Markdown(f"**Solver status:** *{result.solver.status}, {result.solver.termination_condition}*"))
display(Markdown(f"**Optimal solution** for determistic demand equal to $100$: $x = {model.x.value:.1f}$"))
display(Markdown(f"**Optimal deterministic profit:** ${model.total_expected_profit():.0f}$€"))

**Solver status:** *ok, optimal*

**Optimal solution** for determistic demand equal to $100$: $x = 100.0$

**Optimal deterministic profit:** $1500$€

We now assess how well we perform taking the average demand as optimal decision the news vendor problem for each of the three demand distributions above.

(c) For a fixed decision variable $x=100$, approximate the expected profit of the news vendor for each of the three distributions above using the Sample Average Approximation method with $N=2500$ points. More specifically, generate $N=2500$ samples from the considered distribution and solve the extensive form of the stochastic LP resulting from those $N=2500$ scenarios.

In [5]:
# SAA of the two-stage stochastic LP to calculate the expected profit when buying the average

def NaiveNewsvendorSAA(N, sample, distributiontype):

    model = pyo.ConcreteModel()

    def indices_rule(model):
        return range(N)
    model.indices = pyo.Set(initialize=indices_rule)
    model.xi = pyo.Param(model.indices, initialize=dict(enumerate(sample)))

    # first stage variable
    model.x = 100.0 #bought

    def first_stage_profit(model):
        return -c * model.x

    model.first_stage_profit = pyo.Expression(rule=first_stage_profit)

    # second stage variables
    model.y = pyo.Var(model.indices, within=pyo.NonNegativeReals) #sold
    model.z = pyo.Var(model.indices, within=pyo.NonNegativeReals) #unsold to be returned 

    # second stage constraints
    model.cantsoldthingsidonthave = pyo.ConstraintList()
    model.newspapersdonotdisappear = pyo.ConstraintList()
    for i in model.indices:
        model.cantsoldthingsidonthave.add(expr=model.y[i] <= model.xi[i])
        model.newspapersdonotdisappear.add(expr=model.y[i] + model.z[i] == model.x)

    def second_stage_profit(model):
        return sum([s * model.y[i] + r * model.z[i] for i in model.indices])/float(N)

    model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

    def total_profit(model):
        return model.first_stage_profit + model.second_stage_profit

    model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

    result = cbc_solver.solve(model)

    display(Markdown(f"**Approximate expected optimal profit when using the average** $x=100$ with {distributiontype} demand: ${model.total_expected_profit():.2f}$€"))
    return model.total_expected_profit()

np.random.seed(20122020)
N = 2500

samples = np.random.uniform(low=50.0, high=150.0, size=N)
naiveprofit_uniform = NaiveNewsvendorSAA(N, samples, 'uniform')

shape = 2
xm = 50
samples = (np.random.pareto(a=shape, size=N) + 1) *  xm
naiveprofit_pareto = NaiveNewsvendorSAA(N, samples, 'Pareto')

shape=2
scale=113
samples = scale*np.random.weibull(a=shape, size=N)
naiveprofit_weibull = NaiveNewsvendorSAA(N, samples, 'Weibull')

**Approximate expected optimal profit when using the average** $x=100$ with uniform demand: $1245.56$€

**Approximate expected optimal profit when using the average** $x=100$ with Pareto demand: $1009.53$€

**Approximate expected optimal profit when using the average** $x=100$ with Weibull demand: $1088.44$€

(d) Solve approximately the news vendor problem for each of the three distributions above using the Sample Average Approximation method with $N=2500$ points. More specifically, generate $N=2500$ samples from the considered distribution and solve the extensive form of the stochastic LP resulting from those $N=2500$ scenarios. For each of the three distribution, compare the optimal expected profit with that obtained in (c) and calculate the value of the stochastic solution (VSS).

In [6]:
# Two-stage stochastic LP for uniform distribution

c = 10
s = 25
r = 5

def NewsvendorSAA(N, sample, distributiontype):

    model = pyo.ConcreteModel()

    def indices_rule(model):
        return range(N)
    model.indices = pyo.Set(initialize=indices_rule)
    model.xi = pyo.Param(model.indices, initialize=dict(enumerate(sample)))

    # first stage variable
    model.x = pyo.Var(within=pyo.NonNegativeReals) #bought

    def first_stage_profit(model):
        return -c * model.x

    model.first_stage_profit = pyo.Expression(rule=first_stage_profit)

    # second stage variables
    model.y = pyo.Var(model.indices, within=pyo.NonNegativeReals) #sold
    model.z = pyo.Var(model.indices, within=pyo.NonNegativeReals) #unsold to be returned 

    # second stage constraints
    model.cantsoldthingsidonthave = pyo.ConstraintList()
    model.newspapersdonotdisappear = pyo.ConstraintList()
    for i in model.indices:
        model.cantsoldthingsidonthave.add(expr=model.y[i] <= model.xi[i])
        model.newspapersdonotdisappear.add(expr=model.y[i] + model.z[i] == model.x)

    def second_stage_profit(model):
        return sum([s * model.y[i] + r * model.z[i] for i in model.indices])/float(N)

    model.second_stage_profit = pyo.Expression(rule=second_stage_profit)

    def total_profit(model):
        return model.first_stage_profit + model.second_stage_profit

    model.total_expected_profit = pyo.Objective(rule=total_profit, sense=pyo.maximize)

    result = cbc_solver.solve(model)

    display(Markdown(f"**Approximate news vendor problem solution for:** {distributiontype} distribution **using:** $N={N:.0f}$ points"))
    display(Markdown(f"**Solver status:** *{result.solver.status}, {result.solver.termination_condition}*"))
    display(Markdown(f"**Approximate optimal solution:** $x = {model.x.value:.2f}$"))
    display(Markdown(f"**Approximate expected optimal profit:** ${model.total_expected_profit():.2f}$€"))
    return model.total_expected_profit()

np.random.seed(20122020)
N = 2500

samples = np.random.uniform(low=50.0, high=150.0, size=N)
smartprofit_uniform = NewsvendorSAA(N, samples, 'uniform')
display(Markdown(f"**Value of the stochastic solution:** ${smartprofit_uniform:.2f}-{naiveprofit_uniform:.2f} = {smartprofit_uniform-naiveprofit_uniform:.2f}$€"))

shape = 2
xm = 50
samples = (np.random.pareto(a=shape, size=N) + 1) *  xm
smartprofit_pareto = NewsvendorSAA(N, samples, 'Pareto')
display(Markdown(f"**Value of the stochastic solution:** ${smartprofit_pareto:.2f}-{naiveprofit_pareto:.2f} = {smartprofit_pareto-naiveprofit_pareto:.2f}$€"))

shape=2
scale=113
samples = scale*np.random.weibull(a=shape, size=N)
smartprofit_weibull =NewsvendorSAA(N, samples, 'Weibull')
display(Markdown(f"**Value of the stochastic solution:** ${smartprofit_weibull:.2f}-{naiveprofit_weibull:.2f} = {smartprofit_weibull-naiveprofit_weibull:.2f}$€"))

**Approximate news vendor problem solution for:** uniform distribution **using:** $N=2500$ points

**Solver status:** *ok, optimal*

**Approximate optimal solution:** $x = 123.94$

**Approximate expected optimal profit:** $1302.12$€

**Value of the stochastic solution:** $1302.12-1245.56 = 56.56$€

...

**Approximate news vendor problem solution for:** Weibull distribution **using:** $N=2500$ points

**Solver status:** *ok, optimal*

**Approximate optimal solution:** $x = 133.15$

**Approximate expected optimal profit:** $1157.84$€

**Value of the stochastic solution:** $1157.84-1088.44 = 69.40$€